# Model Criticism

"全てのモデルは間違っている"(Box, 1976)と言われるように我々はモデルが本当に正しいかどうかを検証することはできません。
しかしながら我々はモデルがどこで間違っているかを明らかにする　ことができます。
Model criticismはモデルに近似としての正当性を与えるか、あるいは再検討の方向性を与えてくれるものです。

Model criticismは一般には事後予測分布を使って解析されることが多いです。

$ p(\mathbf{x}_\text{new} \mid \mathbf{x}) = \int p(\mathbf{x}_\text{new} \mid \mathbf{z}) p(\mathbf{z} \mid \mathbf{x}) \text{d} \mathbf{z} $

モデルの事後予測分布は新しいデータを過去の観測から生成するのに使われ、その新しいデータで予測をすることができます。
新しいデータの尤度を計算し、事後分布に関する隠れた変数の集合に対して平均をとることでそれを作ることができます。

事後予測をするための有用な関数はcopyです。例としてモデルが事前分布zと結びついて尤度xで定義されるとします。この時予測事後分布は

In [ ]:
x_post = ed.copy(x, {z: qz})

と書かれます。

ここで(確率モデルの)グラフの中の尤度に対応するnode xの事前分布zへの依存性を推定された事後分布qzへの依存へ置き換えています。

さらにModel criticismに関するいくつかのテクニックを説明します。

# 点評価

点評価は学習されたモデルを評価するためのスカラー値を取る計量方法ということができます(Gneiting & Raftery, 2007; Winkler, 1994)。

例えば分類モデルの各観測に対するラベルを予測し、真のラベルと比較することでモデルを評価することができます。
Edwardは分類誤差(classification error)や平均絶対値誤差(mean absolute error)など様々な計量方法を実装しています。

ed.evaluate()メソッドは入力として評価のための計量方法とデータの辞書の組をとります。
推測の結果としてデータの辞書はモデルにおける観測された変数と結びつけられます。

以下の場合これは出力の事後予測ランダム変数y_postとy_trainとを、入力のプレースホルダーxとx_trainとを結びつけます。

In [ ]:
ed.evaluate('categorical_accuracy', data={y_post: y_train, x: x_train})
ed.evaluate('mean_absolute_error', data={y_post: y_train, x: x_train})

点評価は教師なし学習にも適用することができます。例えばデータが観測される尤度を評価することができます。

ed.evaluate('log_likelihood', data={x_post: x_train})

学習で使わずにとっておいたデータを使ってモデルを評価することは一般によく行われます。　

これをするためにはまず取り置かれた(held-out)データに対応した全ての隠れた変数を推測し、大域的な変数を固定する必要があります。以下ではこの実例を示します。その後取り置かれたデータに対して予測を行います。

In [ ]:
from edward.models import Categorical

#テストデータに対する局所的な事後分布の因子qz_testを作る。テストデータはN_test個のデータ点を持つ。
qz_test = Categorical(logits=tf.Variable(tf.zeros[N_test, K]))

#大域的な因子(qbeta)に基づいた局所的な因子の推測を行う。
# run local inference conditional on global factors
inference_test = ed.Inference({z: qz_test}, data={x: x_test, beta: qbeta})
inference_test.run()

#テストデータに対する事後予測を行う。
x_post = ed.copy(x, {z: qz_test, beta: qbeta}})
ed.evaluate('log_likelihood', data={x_post: x_test})

点評価は形式的には決定理論における　scoring rulesとして知られています。scoring rulesはモデル比較、モデル選択、そしてモデル間平均に有用な概念です。

さらなる詳細はcriticism APIを参照してください。点評価の例はチュートリアルの[教師あり学習(回帰)](BayesianLinearRegression.ipynb)にあります。

# 事後予測の確認(PPC)

事後予測の確認(Posterior predictive checks: PPCs)はモデルから生成されたデータが真の分布から生成されたデータからどの程度離れているかを調べる方法です。数値的、あるいは図示することでその度合いを評価することができます。
PPCは点評価の確率分布に一般化したものと考えることができます(Box, 1980; Gelman, Meng, & Stern, 1996; Meng, 1994; Rubin, 1984)。

最も単純なPPCは$T(\mathbf{x}_\text{new}) = \max(\mathbf{x}_\text{new})T(x)$
のように検定量を事後予測で作られたデータに対して適用することでできます。

$T(\mathbf{x}_\text{new})$を多くのデータの複製に対して適用することで分布が作られます。

この作られた分布と真のデータの検定量を比較します。

![a](http://edwardlib.org/images/ppc.png)

上の図では$T(x)$はこの参照確率分布における低い領域にあります。もしモデルが正しければ検定量を観測する確率はとても低いでしょう。
これはモデルがデータの分布にちゃんとあっていないということが確認できた　　モデルに改善の余地があるということを示します。

より一般的には検定量はdiscrepancy　functionとして知られるモデルの隠れた変数zの関数$T(\mathbf{x}, \mathbf{z})$です。

discrepancyの例としては点評価で使われた計量があります。
つまり点評価は実際のデータxに対して参照分布を使わずに単純に$T(\mathbf{x}, \mathbf{z})$を計算するというPPCの特殊な例であるということができます。
参照分布を使うことで分布全体の点に対して確率的な主張を与えることができます。

ed.ppc()メソッドは様々なdiscrepancy　functionを計算するための基礎になります。このメソッドはユーザーが定義したdiscrepancyとデータの辞書を入力とします。

In [ ]:
ed.ppc(lambda xs, zs: tf.reduce_mean(xs[x_post]), data={x_post: x_train})

discrepancyは隠れた変数を入力として取ることもでき、それはPPCへと渡されます。

In [ ]:
ed.ppc(lambda xs, zs: tf.maximum(zs[z]),
       data={y_post: y_train, x_ph: x_train},
       latent_vars={z: qz, beta: qbeta})

さらには[Criticism API](http://edwardlib.org/api/criticism)を参照

PPCはある段階でのモデルをデータに合わせるために単純化あるいは拡張すべきかどうかを決めるための優れたツールです。

PPCは古典的仮説検定に着想を得ており、モデル評価の方法は多くのサンプルを評価に用いる頻度主義的な見方に基づいています。

PPCは仮説検定、モデル比較、モデル選択、モデル平均化にも用いることができます。特にベイズ仮説検定の形で使われることがあるのは重要です。

仮説検定は一般には推奨されません。単一の検定からの２値(yes or no)の意志決定は一般には信頼に足るしよう事例とは言えません。
我々はモデルのデータに対する適合を全体的に理解するために多くのPPCを使うことを推奨します。

# References

- Box, G. E. (1976). Science and statistics. Journal of the American Statistical Association, 71(356), 791–799.

- Box, G. E. (1980). Sampling and Bayes’ inference in scientific modelling and robustness. Journal of the Royal Statistical Society. Series A (General), 383–430.

- Gelman, A., Meng, X.-L., & Stern, H. (1996). Posterior predictive assessment of model fitness via realized discrepancies. Statistica Sinica, 733–760.

- Gneiting, T., & Raftery, A. E. (2007). Strictly proper scoring rules, prediction, and estimation. Journal of the American Statistical Association, 102(477), 359–378.

- Meng, X.-L. (1994). Posterior predictive pp-values. The Annals of Statistics, 1142–1160.

- Rubin, D. B. (1984). Bayesianly justifiable and relevant frequency calculations for the applied statistician. The Annals of Statistics, 12(4), 1151–1172.

- Winkler, R. L. (1994). Evaluating probabilities: Asymmetric scoring rules. Management Science, 40(11), 1395–1405.